In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from xmitgcm import utils
from xmitgcm import open_mdsdataset
from xgcm import Grid
from xhistogram.xarray import histogram
from dask.diagnostics import ProgressBar

In [7]:
import sys
sys.path.append("scripts")
from filesystem import *

In [8]:
ds, grid = load_RT_canyon_hydrostatic()

In [9]:
release_time = np.datetime64(ds.release_time)

# Small chunks to make xhistogram happy
ds = ds.chunk({'Z':10})

In [10]:
dS = 0.001
dT = 0.01
Sbins = np.arange(34.9, 35.1+dS/2., dS)
Tbins = np.arange(3.25, 5.25+dT/2., dT)
dV = ds['drF']*ds['rA']*ds['hFacC']

tracer_hist = histogram(ds.SALT, ds.THETA, bins=[Sbins, Tbins], dim=['XC', 'YC', 'Z'], weights=ds['TRAC01']*dV, density=True)

In [11]:
with ProgressBar():
    tracer_hist = tracer_hist.compute()

[#####                                   ] | 14% Completed |  6min 16.9s

IOStream.flush timed out


[########################################] | 100% Completed | 28min  8.6s


In [12]:
first_moment_T = (ds['THETA']*ds['TRAC01']*dV).sum(['XC', 'YC', 'Z'])/(ds['TRAC01']*dV).sum(['XC', 'YC', 'Z'])
first_moment_S = (ds['SALT']*ds['TRAC01']*dV).sum(['XC', 'YC', 'Z'])/(ds['TRAC01']*dV).sum(['XC', 'YC', 'Z'])
first_moment_XC = (ds['XC']*ds['TRAC01']*dV).sum(['XC', 'YC', 'Z'])/(ds['TRAC01']*dV).sum(['XC', 'YC', 'Z'])
first_moment_YC = (ds['YC']*ds['TRAC01']*dV).sum(['XC', 'YC', 'Z'])/(ds['TRAC01']*dV).sum(['XC', 'YC', 'Z'])
first_moment_Z = (ds['Z']*ds['TRAC01']*dV).sum(['XC', 'YC', 'Z'])/(ds['TRAC01']*dV).sum(['XC', 'YC', 'Z'])


In [ ]:
with ProgressBar():
    first_moment_T = first_moment_T.compute()
    first_moment_S = first_moment_S.compute()
    first_moment_XC = first_moment_XC.compute()
    first_moment_YC = first_moment_YC.compute()
    first_moment_Z = first_moment_Z.compute()
    TRAC01_max = ds['TRAC01'].max(['XC', 'YC', 'Z'])
    TRAC01_argmax = ds['TRAC01'].argmax(['XC', 'YC', 'Z'])
    XC_max = ds['XC'][TRAC01_argmax['XC'].compute()]
    YC_max = ds['YC'][TRAC01_argmax['YC'].compute()]
    Z_max = ds['Z'][TRAC01_argmax['Z'].compute()]

[########################################] | 100% Completed | 11min  1.5s
[###                                     ] | 7% Completed | 58.2s

In [ ]:
out_ds = xr.Dataset({
    'TR01_TSMASSPDF': tracer_hist,
    'THETA_bar': first_moment_T,
    'SALT_bar': first_moment_S,
    'XC_bar': first_moment_XC,
    'YC_bar': first_moment_YC,
    'Z_bar': first_moment_Z,
    'XC_max': XC_max,
    'YC_max': YC_max,
    'Z_max': Z_max
})
out_ds.to_netcdf("data/tracer_stats.nc")
out_ds.close()